# 1.30i: Dyadic Lattice Verification

**Purpose:** Verify that lattice displacements are exactly dyadic (integers + negative powers of 2).

## Hypothesis

If the bfloat16 quantization is working correctly, every coordinate displacement should be:
1. **Exact integer:** 0, 1, 2, 3, ...
2. **Integer + dyadic fraction:** n + 2^(-k) where n ∈ ℤ, k ∈ {1, 2, 3, ...}
   - Examples: 0.5, 1.5, 2.25, 3.125, ...

**There should be NOTHING ELSE.**

## Questions

1. How many are **exactly** integers? (Not within epsilon—bitwise exact)
2. How many are within epsilon of an integer?
3. How many have fractional parts that are **exactly** negative powers of 2?
4. **How many are OTHERS?** (The scary category)

Then: if there are others, what the hell are they?

## Parameters

In [12]:
# Data path
DATA_PATH = "../tensors/Thimble/thimble_6.h5"

# Epsilon for "close to" comparisons
EPSILON = 1e-10

# Negative powers of 2 to check
# 2^-1 = 0.5, 2^-2 = 0.25, 2^-3 = 0.125, ..., 2^-10 ≈ 0.001
DYADIC_FRACTIONS = [2**(-k) for k in range(1, 11)]  # [0.5, 0.25, 0.125, ...]

print("✓ Parameters set")
print(f"\nDyadic fractions to check: {DYADIC_FRACTIONS[:5]} ... (10 total)")

✓ Parameters set

Dyadic fractions to check: [0.5, 0.25, 0.125, 0.0625, 0.03125] ... (10 total)


## Imports

In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path
from collections import Counter

print("✓ Imports complete")

✓ Imports complete


## Load Data (Keep in bfloat16!)

In [14]:
print(f"Loading dead token trajectories from {DATA_PATH}...\n")

with h5py.File(DATA_PATH, 'r') as f:
    n_steps = f.attrs['num_steps']
    dead_mask = torch.from_numpy(f['dead_mask'][:]).bool()
    n_dead = dead_mask.sum().item()
    
    print(f"Loading W[:, dead_mask, :] ... ({n_steps+1} × {n_dead} × 64)")
    W_dead = torch.from_numpy(f['W'][:, dead_mask, :]).to(torch.bfloat16)

print(f"\n✓ Data loaded: {tuple(W_dead.shape)}")
print(f"Dtype: {W_dead.dtype}")

Loading dead token trajectories from ../tensors/Thimble/thimble_6.h5...

Loading W[:, dead_mask, :] ... (6001 × 3699 × 64)

✓ Data loaded: (6001, 3699, 64)
Dtype: torch.bfloat16


## Compute ULP and Lattice Displacements

In [15]:
def compute_ulp(x):
    """Compute ULP for bfloat16 values."""
    ulp_pos = torch.nextafter(x, x + torch.ones_like(x)) - x
    ulp_neg = x - torch.nextafter(x, x - torch.ones_like(x))
    ulp = torch.where(x >= 0, ulp_pos, ulp_neg)
    return ulp.abs()

print("Computing lattice-space displacements...\n")

U_start = compute_ulp(W_dead[:-1])
dW = W_dead[1:] - W_dead[:-1]
dW_lattice_bf16 = dW / (U_start + 1e-30)
dW_lattice = dW_lattice_bf16.float()

print(f"Lattice displacements: {tuple(dW_lattice.shape)}")
print(f"✓ Lattice transformation complete")

Computing lattice-space displacements...

Lattice displacements: (6000, 3699, 64)
✓ Lattice transformation complete


## Classification: Integers vs. Dyadic Fractions vs. Others

In [16]:
print("Classifying coordinates...\n")

# Flatten to all coordinates
coords = dW_lattice.flatten()
n_total = len(coords)

print(f"Total coordinates: {n_total:,}\n")

# Separate integer and fractional parts
int_part = coords.floor()
frac_part = coords - int_part  # Always in [0, 1)

# 1. Exact integers (fractional part == 0 exactly)
exact_int = (frac_part == 0)
n_exact_int = exact_int.sum().item()

# 2. Close to integer (within epsilon)
close_to_int = (frac_part < EPSILON) | (frac_part > (1 - EPSILON))
n_close_int = close_to_int.sum().item()

# 3. Dyadic fractions (fractional part is exactly 2^-k for some k)
# Check if fractional part matches any dyadic fraction
is_dyadic = torch.zeros_like(frac_part, dtype=torch.bool)
for dyadic in DYADIC_FRACTIONS:
    # Check both x.f and 1-x.f (since 0.5 could appear as 0.5 or as next_int - 0.5)
    matches_dyadic = (torch.abs(frac_part - dyadic) < EPSILON) | (torch.abs(frac_part - (1 - dyadic)) < EPSILON)
    is_dyadic = is_dyadic | matches_dyadic

n_dyadic = is_dyadic.sum().item()

# 4. OTHERS (the scary category)
is_other = ~close_to_int & ~is_dyadic
n_other = is_other.sum().item()

# Verification: should sum to n_total
# (close_to_int includes exact_int, so we count separately)
n_check = n_close_int + n_dyadic + n_other

print("=" * 80)
print("DYADIC LATTICE VERIFICATION")
print("=" * 80 + "\n")

print(f"1. EXACT INTEGERS (frac == 0.0 exactly):")
print(f"   Count:    {n_exact_int:,}")
print(f"   Fraction: {n_exact_int/n_total:.4%}")
print()

print(f"2. CLOSE TO INTEGER (|frac| < {EPSILON} or |frac-1| < {EPSILON}):")
print(f"   Count:    {n_close_int:,}")
print(f"   Fraction: {n_close_int/n_total:.4%}")
print(f"   (includes exact integers)")
print()

print(f"3. DYADIC FRACTIONS (frac = 2^-k for k=1..10):")
print(f"   Count:    {n_dyadic:,}")
print(f"   Fraction: {n_dyadic/n_total:.4%}")
print()

print(f"4. OTHERS (not integer, not dyadic):")
print(f"   Count:    {n_other:,}")
print(f"   Fraction: {n_other/n_total:.4%}")
print()

print("-" * 80)
print(f"Verification: {n_close_int} + {n_dyadic} + {n_other} = {n_check:,}")
if n_check == n_total:
    print(f"✓ Sum matches total: {n_total:,}")
else:
    print(f"✗ ERROR: Sum = {n_check:,}, expected {n_total:,} (diff: {n_total - n_check:,})")
print("=" * 80)

Classifying coordinates...

Total coordinates: 1,420,416,000

DYADIC LATTICE VERIFICATION

1. EXACT INTEGERS (frac == 0.0 exactly):
   Count:    1,420,199,867
   Fraction: 99.9848%

2. CLOSE TO INTEGER (|frac| < 1e-10 or |frac-1| < 1e-10):
   Count:    1,420,199,867
   Fraction: 99.9848%
   (includes exact integers)

3. DYADIC FRACTIONS (frac = 2^-k for k=1..10):
   Count:    216,133
   Fraction: 0.0152%

4. OTHERS (not integer, not dyadic):
   Count:    0
   Fraction: 0.0000%

--------------------------------------------------------------------------------
Verification: 1420199867 + 216133 + 0 = 1,420,416,000
✓ Sum matches total: 1,420,416,000


## Breakdown of Dyadic Fractions

In [17]:
if n_dyadic > 0:
    print("\n" + "=" * 80)
    print("DYADIC FRACTION BREAKDOWN")
    print("=" * 80 + "\n")
    
    print("Count by specific dyadic value:\n")
    
    for k, dyadic in enumerate(DYADIC_FRACTIONS, start=1):
        # Check matches (both x.f and 1-x.f)
        matches = (torch.abs(frac_part - dyadic) < EPSILON) | (torch.abs(frac_part - (1 - dyadic)) < EPSILON)
        n_matches = matches.sum().item()
        
        if n_matches > 0:
            print(f"  2^-{k} = {dyadic:8.6f}:  {n_matches:12,}  ({n_matches/n_total:.4%})")
    
    print("\n" + "=" * 80)
else:
    print("\nNo dyadic fractions found.")


DYADIC FRACTION BREAKDOWN

Count by specific dyadic value:

  2^-1 = 0.500000:       216,133  (0.0152%)



## Investigation: What Are The Others?

In [18]:
if n_other > 0:
    print("\n" + "=" * 80)
    print("⚠ INVESTIGATION: THE OTHERS")
    print("=" * 80 + "\n")
    
    print(f"Found {n_other:,} coordinates that are NOT integers or dyadic fractions.\n")
    
    # Get the fractional parts of "others"
    other_fracs = frac_part[is_other].numpy()
    
    print("Statistics of unexplained fractional parts:")
    print("-" * 40)
    print(f"  Min:    {other_fracs.min():.10f}")
    print(f"  Max:    {other_fracs.max():.10f}")
    print(f"  Mean:   {other_fracs.mean():.10f}")
    print(f"  Median: {np.median(other_fracs):.10f}")
    print(f"  Std:    {other_fracs.std():.10f}")
    
    # Show a sample
    print("\nSample of 20 unexplained values (full coordinates):")
    print("-" * 40)
    other_coords = coords[is_other].numpy()
    sample_size = min(20, len(other_coords))
    sample = np.random.choice(other_coords, size=sample_size, replace=False)
    for i, val in enumerate(sorted(sample), 1):
        frac = val - np.floor(val)
        print(f"  {i:2d}. {val:12.8f}  (frac = {frac:.8f})")
    
    # Histogram of fractional parts
    fig, ax = plt.subplots(figsize=(12, 6), dpi=200)
    ax.hist(other_fracs, bins=100, color='red', alpha=0.7, edgecolor='black', linewidth=0.5)
    
    # Mark dyadic fractions for reference
    for dyadic in DYADIC_FRACTIONS[:5]:  # Show first 5
        ax.axvline(dyadic, color='blue', linestyle='--', linewidth=1, alpha=0.5)
    
    ax.set_xlabel('Fractional Part')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of "Other" Fractional Parts (NOT Integer, NOT Dyadic)')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "=" * 80)
else:
    print("\n" + "=" * 80)
    print("✓ NO OTHERS FOUND")
    print("=" * 80 + "\n")
    print("All coordinates are either integers or dyadic fractions.")
    print("Data is perfectly quantized to the dyadic lattice!")
    print("\n" + "=" * 80)


✓ NO OTHERS FOUND

All coordinates are either integers or dyadic fractions.
Data is perfectly quantized to the dyadic lattice!



## Alpha's Curiosity Corner

In [19]:
print("\n" + "=" * 80)
print("ALPHA'S CURIOSITY CORNER")
print("=" * 80 + "\n")

# 1. Most common fractional values
print("Most common fractional parts (top 10):\n")
frac_rounded = (frac_part * 1000).round() / 1000  # Round to 3 decimals for counting
frac_counter = Counter(frac_rounded.numpy())
for frac, count in frac_counter.most_common(10):
    print(f"  {frac:6.3f}:  {count:12,}  ({count/n_total:.4%})")

# 2. Coordinate magnitude distribution
print("\n" + "-" * 80)
print("\nCoordinate magnitude distribution:\n")
coord_abs = coords.abs().numpy()
print(f"  Mean:   {coord_abs.mean():.4f} ULP")
print(f"  Median: {np.median(coord_abs):.4f} ULP")
print(f"  Max:    {coord_abs.max():.4f} ULP")
print(f"  90th:   {np.percentile(coord_abs, 90):.4f} ULP")
print(f"  99th:   {np.percentile(coord_abs, 99):.4f} ULP")

# 3. How many coords are zero?
print("\n" + "-" * 80)
print("\nZero displacement analysis:\n")
n_zero = (coords == 0).sum().item()
print(f"  Exactly zero:  {n_zero:,}  ({n_zero/n_total:.4%})")
print(f"  Non-zero:      {n_total - n_zero:,}  ({(n_total - n_zero)/n_total:.4%})")

# 4. Sign distribution (for non-zero)
print("\n" + "-" * 80)
print("\nSign distribution (non-zero coords):\n")
nonzero_coords = coords[coords != 0]
if len(nonzero_coords) > 0:
    n_pos = (nonzero_coords > 0).sum().item()
    n_neg = (nonzero_coords < 0).sum().item()
    print(f"  Positive:  {n_pos:,}  ({n_pos/len(nonzero_coords):.4%})")
    print(f"  Negative:  {n_neg:,}  ({n_neg/len(nonzero_coords):.4%})")
    print(f"  Ratio (pos/neg): {n_pos/n_neg:.4f}")

print("\n" + "=" * 80)
print("✓ Curiosity satisfied.")
print("=" * 80)


ALPHA'S CURIOSITY CORNER

Most common fractional parts (top 10):

   0.000:  1,420,199,867  (99.9848%)
   0.500:       216,133  (0.0152%)

--------------------------------------------------------------------------------

Coordinate magnitude distribution:

  Mean:   15033100218558054400.0000 ULP
  Median: 0.0000 ULP
  Max:    1044511908147039606946136064.0000 ULP
  90th:   0.0000 ULP
  99th:   2.0000 ULP

--------------------------------------------------------------------------------

Zero displacement analysis:

  Exactly zero:  1,385,749,994  (97.5594%)
  Non-zero:      34,666,006  (2.4406%)

--------------------------------------------------------------------------------

Sign distribution (non-zero coords):

  Positive:  18,896,417  (54.5099%)
  Negative:  15,769,589  (45.4901%)
  Ratio (pos/neg): 1.1983

✓ Curiosity satisfied.


## Final Verdict

In [20]:
print("\n" + "=" * 80)
print("FINAL VERDICT")
print("=" * 80 + "\n")

dyadic_total = n_close_int + n_dyadic
dyadic_frac = dyadic_total / n_total

print(f"Coordinates on dyadic lattice: {dyadic_total:,} / {n_total:,} ({dyadic_frac:.6%})")
print(f"Unexplained coordinates: {n_other:,} ({n_other/n_total:.6%})\n")

if n_other == 0:
    print("✓✓✓ PERFECT DYADIC LATTICE ✓✓✓")
    print("\nAll coordinates are integers or dyadic fractions.")
    print("The bfloat16 quantization is working exactly as expected.")
    print("Safe to proceed with integer + dyadic fraction analysis.")
elif dyadic_frac > 0.9999:  # >99.99%
    print("✓ ESSENTIALLY DYADIC")
    print(f"\n{dyadic_frac:.4%} of coords are on dyadic lattice.")
    print(f"Only {n_other:,} unexplained ({n_other/n_total:.6%}).")
    print("Probably safe to round/ignore the others.")
elif dyadic_frac > 0.99:  # >99%
    print("⚠ MOSTLY DYADIC")
    print(f"\n{dyadic_frac:.2%} dyadic, but {n_other:,} others need investigation.")
else:
    print("✗ NOT DYADIC")
    print(f"\nOnly {dyadic_frac:.2%} on dyadic lattice.")
    print(f"{n_other:,} unexplained coordinates—something is wrong.")

print("\n" + "=" * 80)
print("✓ Dyadic lattice verification complete.")
print("=" * 80)


FINAL VERDICT

Coordinates on dyadic lattice: 1,420,416,000 / 1,420,416,000 (100.000000%)
Unexplained coordinates: 0 (0.000000%)

✓✓✓ PERFECT DYADIC LATTICE ✓✓✓

All coordinates are integers or dyadic fractions.
The bfloat16 quantization is working exactly as expected.
Safe to proceed with integer + dyadic fraction analysis.

✓ Dyadic lattice verification complete.
